In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import pandas as pd 
import numpy as np
import os

#import data 
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/GRAD-C24_Machine_Learning/MLProject_KenyaFinancial
DATA_PATH = "/content/drive/MyDrive/GRAD-C24_Machine_Learning/MLProject_KenyaFinancial/raw_data/"
os.getcwd()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/GRAD-C24_Machine_Learning/MLProject_KenyaFinancial


'/content/drive/MyDrive/GRAD-C24_Machine_Learning/MLProject_KenyaFinancial'

In [2]:
from os import listdir
FILE_LIST = listdir(DATA_PATH)
print(FILE_LIST)

df_dic={} # create a dictionary with names of datasets as keys and datasets as values
for i, e in enumerate(FILE_LIST):
  df_dic[str(FILE_LIST[i].split(".csv")[0])]= pd.read_csv("raw_data/" + e)

# test
#df_dic['diaries_transactions_all'].columns

['con_prod_hh_over_trx_yr_mo_usd.csv', 'con_pur_hh_by_trx_yr_mo_usd.csv', 'demo_hh.csv', 'demo_ind.csv', 'diaries_goingson_hh.csv', 'diaries_wellbeing_ind.csv', 'edu_hh.csv', 'edu_ind.csv', 'housing_conditions_ownership.csv', 'inc_hh_by_trx_yr_mo_usd_incRR.csv', 'inc_ind_by_trx_yr_mo_usd_incRR.csv', 'pov_hh_by_trx_yr_mo_usd_incRR.csv', 'rem_hh_by_trx_yr_mo_usd.csv', 'pov_hh_by_trx_yr_mo_usd_excRR.csv', 'diaries_transactions_all.csv']


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


## Start merging
#### first we merge datasets with same keys together, then we merge the bigger (already merged) datasets together

In [3]:
from functools import reduce

dfs = [df_dic['diaries_transactions_all'], df_dic['demo_hh'] , df_dic['con_prod_hh_over_trx_yr_mo_usd'], df_dic['edu_hh']]

result_hhids = reduce(lambda df_left, df_right: pd.merge(df_left, df_right, 
                                              right_on = 'hh_ids', left_on = 'hh_ids', 
                                              how='outer'), 
                  dfs)

In [4]:
result_hhids.head()

,hh_ids,unique_hhs,first_trx_date_hh,last_trx_date_hh,tot_hh_daysofobs,tot_hh_monthsofobs,interview_designation,int_date,int_month,int_year,...,dem_pae_oecd,dem_pae_kihbs,con_prod_hh_medpti,edu_hh_child613inschool,edu_hh_child1417inschool,edu_hh_girls613inschool,edu_hh_girls1417inschool,edu_hh_boys613inschool,edu_hh_boys1417inschool,edu_hh_headattain_yrs
0,KELDL02,NaN,03sep2012,02oct2013,394,13,04=Diaries Interview,02oct2013,10,2013,...,2.7,2.48,0.0,NaN,NaN,NaN,NaN,NaN,NaN,12.0
1,KELDL02,NaN,03sep2012,02oct2013,394,13,04=Diaries Interview,13dec2012,12,2012,...,2.7,2.48,0.0,NaN,NaN,NaN,NaN,NaN,NaN,12.0
2,KELDL02,NaN,03sep2012,02oct2013,394,13,04=Diaries Interview,19may2013,5,2013,...,2.7,2.48,0.0,NaN,NaN,NaN,NaN,NaN,NaN,12.0
3,KELDL02,NaN,03sep2012,02oct2013,394,13,04=Diaries Interview,02oct2013,10,2013,...,2.7,2.48,0.0,NaN,NaN,NaN,NaN,NaN,NaN,12.0
4,KELDL02,NaN,03sep2012,02oct2013,394,13,04=Diaries Interview,16jun2013,6,2013,...,2.7,2.48,0.0,NaN,NaN,NaN,NaN,NaN,NaN,12.0


In [5]:
dfs = [df_dic['con_pur_hh_by_trx_yr_mo_usd'], df_dic['inc_hh_by_trx_yr_mo_usd_incRR'], df_dic['pov_hh_by_trx_yr_mo_usd_excRR'], df_dic['pov_hh_by_trx_yr_mo_usd_incRR'], df_dic['rem_hh_by_trx_yr_mo_usd']]

result_hhids_trxyrmo = reduce(lambda df_left, df_right: pd.merge(df_left, df_right, 
                                              right_on = ['hh_ids', 'trx_yr_mo'], left_on =['hh_ids', 'trx_yr_mo'], 
                                              how= 'outer'),
                  dfs)

In [6]:
result_hhids_trxyrmo.head()

,hh_ids,trx_yr_mo,con_pur_hh_byti_x,con_pur_hh_byti_ALC,con_pur_hh_byti_CLN,con_pur_hh_byti_CLTH,con_pur_hh_byti_COMM,con_pur_hh_byti_EDU,con_pur_hh_byti_ENRGY,con_pur_hh_byti_ENT,...,rem_hh_net_byti_cash,rem_hh_count_byti_inkind,rem_hh_abs_byti_inkind,rem_hh_net_byti_inkind,rem_hh_count_byti_mm,rem_hh_abs_byti_mm,rem_hh_net_byti_mm,rem_hh_count_byti_other,rem_hh_abs_byti_other,rem_hh_net_byti_other
0,KELDK01,2012_10,17.29412,0.0,0.000000,8.235294,0.470588,0.235294,3.529412,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,KELDK01,2012_11,40.52941,0.0,0.000000,0.000000,0.941176,0.000000,10.823530,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,KELDK01,2012_12,42.43530,0.0,1.058824,7.058824,0.588235,0.000000,4.588235,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,KELDK01,2013_01,48.23529,0.0,0.000000,0.000000,3.529412,7.058824,3.764706,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,KELDK01,2013_02,107.52940,0.0,0.000000,0.000000,8.235294,0.000000,14.588240,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
dfs = [df_dic['diaries_wellbeing_ind'], df_dic['diaries_goingson_hh']]

result_hhids_intdate = reduce(lambda df_left, df_right: pd.merge(df_left, df_right, 
                                              right_on = ['hh_ids', 'int_date'], left_on =['hh_ids', 'int_date'], 
                                              how= 'outer'),
                  dfs)


In [8]:
result_hhids_intdate.head()

,hh_ids,m_ids,int_date,wellbeing_q1,wellbeing_q2,wellbeing_q3,wellbeing_q4,goingson_q1,goingson_q2,goingson_q3,goingson_q4,goingson_q5,goingson_q6,goingson_q7,goingson_q8,goingson_q9,goingson_q10
0,KNBOK41,5.010000e+16,26nov2012,Very happy,Very badly,Very well,Very confident,Yes,No,No,No,No,No,No,No,No,NaN
1,KNBOK41,5.010000e+16,10dec2012,Neither happy nor unhappy,Moderately well,Very well,Very confident,No,No,Yes,No,No,No,No,No,No,NaN
2,KNBOK41,5.010000e+16,21jan2013,Very happy,Moderately well,Very well,Very confident,No,No,No,No,No,No,No,No,No,NaN
3,KNBOK41,5.010000e+16,05feb2013,Neither happy nor unhappy,Neither well nor badly,Very well,Very confident,No,No,No,No,No,Yes,No,No,No,NaN
4,KNBOK41,5.010000e+16,19feb2013,Very happy,Moderately well,Moderately well,Very confident,No,No,No,No,No,No,No,No,No,No


In [9]:
dfs = [df_dic['demo_ind'], df_dic['edu_ind']]

result_hhids_inds = reduce(lambda df_left, df_right: pd.merge(df_left, df_right, 
                                              right_on = ['hh_ids', 'm_ids'], left_on =['hh_ids', 'm_ids'], 
                                              how= 'outer'),
                  dfs)

result_hhids_inds['m_ids'] = result_hhids_inds['m_ids'].astype(str) # convert to string for merging purposes

In [10]:
result_hhids_inds.head() 

,hh_ids,m_ids,dem_i_male,dem_i_age_yrs,dem_i_age_5yrgrp,dem_i_age_workage,dem_i_relhead,dem_i_hhead,dem_i_marstat,dem_i_tribe,dem_i_children_tot,dem_i_children_inhh,edu_i_enrolled,edu_i_attain,edu_i_attain_yrs
0,KELDK01,56134392820800000,Female,8,[5 - 10),0,Son or daughter,0,Never married/lived together,Kalenjin,NaN,NaN,1.0,Primary (some or complete),3.0
1,KELDK01,56134390476100000,Male,30,[30 - 35),1,Household head,1,Married/living together,Kalenjin,2.0,0.0,0.0,Primary (some or complete),8.0
2,KELDK01,56134392826900000,Male,2,[0 - 5),0,Son or daughter,0,Never married/lived together,Kalenjin,NaN,NaN,0.0,No education,0.0
3,KELDK01,56134390654900000,Female,24,[20 - 25),1,Husband or wife,0,Married/living together,Kalenjin,2.0,0.0,0.0,Secondary (some or complete),10.0
4,KELDK02,56134389772300000,Female,35,[35 - 40),1,Husband or wife,0,Married/living together,Luhya,1.0,1.0,0.0,Primary (some or complete),2.0


In [11]:
# final part of merging

official_1 = pd.merge(result_hhids, result_hhids_trxyrmo, right_on = ['hh_ids', 'trx_yr_mo'], left_on =['hh_ids', 'trx_yr_mo'], 
                                              how= 'outer')



In [12]:
official_2 = pd.merge(official_1, result_hhids_intdate, right_on = ['hh_ids', 'int_date'], left_on =['hh_ids', 'int_date'], 
                                              how= 'outer')

In [13]:
official= pd.merge(official_2, result_hhids_inds, right_on = ['hh_ids', 'm_ids'], left_on =['hh_ids', 'm_ids_trx'], 
                                              how= 'outer')


In [14]:
official.head()

,hh_ids,unique_hhs,first_trx_date_hh,last_trx_date_hh,tot_hh_daysofobs,tot_hh_monthsofobs,interview_designation,int_date,int_month,int_year,...,dem_i_age_workage,dem_i_relhead,dem_i_hhead,dem_i_marstat,dem_i_tribe,dem_i_children_tot,dem_i_children_inhh,edu_i_enrolled,edu_i_attain,edu_i_attain_yrs
0,KELDL02,NaN,03sep2012,02oct2013,394.0,13.0,04=Diaries Interview,02oct2013,10.0,2013.0,...,1.0,Household head,1.0,Married/living together,Luo,2.0,0.0,0.0,Secondary (some or complete),12.0
1,KELDL02,NaN,03sep2012,02oct2013,394.0,13.0,04=Diaries Interview,02oct2013,10.0,2013.0,...,1.0,Household head,1.0,Married/living together,Luo,2.0,0.0,0.0,Secondary (some or complete),12.0
2,KELDL02,NaN,03sep2012,02oct2013,394.0,13.0,04=Diaries Interview,02oct2013,10.0,2013.0,...,1.0,Household head,1.0,Married/living together,Luo,2.0,0.0,0.0,Secondary (some or complete),12.0
3,KELDL02,NaN,03sep2012,02oct2013,394.0,13.0,04=Diaries Interview,02oct2013,10.0,2013.0,...,1.0,Household head,1.0,Married/living together,Luo,2.0,0.0,0.0,Secondary (some or complete),12.0
4,KELDL02,NaN,03sep2012,02oct2013,394.0,13.0,04=Diaries Interview,02oct2013,10.0,2013.0,...,1.0,Household head,1.0,Married/living together,Luo,2.0,0.0,0.0,Secondary (some or complete),12.0


In [15]:
official[official['hh_ids'] == 'KELDK01'] # check 

,hh_ids,unique_hhs,first_trx_date_hh,last_trx_date_hh,tot_hh_daysofobs,tot_hh_monthsofobs,interview_designation,int_date,int_month,int_year,...,dem_i_age_workage,dem_i_relhead,dem_i_hhead,dem_i_marstat,dem_i_tribe,dem_i_children_tot,dem_i_children_inhh,edu_i_enrolled,edu_i_attain,edu_i_attain_yrs
368685,KELDK01,NaN,18sep2012,02oct2013,379.0,13.0,04=Diaries Interview,07aug2013,8.0,2013.0,...,1.0,Household head,1.0,Married/living together,Kalenjin,2.0,0.0,0.0,Primary (some or complete),8.0
368686,KELDK01,NaN,18sep2012,02oct2013,379.0,13.0,04=Diaries Interview,07aug2013,8.0,2013.0,...,1.0,Household head,1.0,Married/living together,Kalenjin,2.0,0.0,0.0,Primary (some or complete),8.0
368687,KELDK01,NaN,18sep2012,02oct2013,379.0,13.0,04=Diaries Interview,07aug2013,8.0,2013.0,...,1.0,Household head,1.0,Married/living together,Kalenjin,2.0,0.0,0.0,Primary (some or complete),8.0
368688,KELDK01,NaN,18sep2012,02oct2013,379.0,13.0,04=Diaries Interview,07aug2013,8.0,2013.0,...,1.0,Household head,1.0,Married/living together,Kalenjin,2.0,0.0,0.0,Primary (some or complete),8.0
368689,KELDK01,NaN,18sep2012,02oct2013,379.0,13.0,04=Diaries Interview,07aug2013,8.0,2013.0,...,1.0,Household head,1.0,Married/living together,Kalenjin,2.0,0.0,0.0,Primary (some or complete),8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
370180,KELDK01,NaN,18sep2012,02oct2013,379.0,13.0,04=Diaries Interview,11oct2012,10.0,2012.0,...,0.0,Son or daughter,0.0,Never married/lived together,Kalenjin,NaN,NaN,0.0,No education,0.0
370181,KELDK01,NaN,18sep2012,02oct2013,379.0,13.0,04=Diaries Interview,25jun2013,6.0,2013.0,...,0.0,Son or daughter,0.0,Never married/lived together,Kalenjin,NaN,NaN,0.0,No education,0.0
370182,KELDK01,NaN,18sep2012,02oct2013,379.0,13.0,04=Diaries Interview,06apr2013,4.0,2013.0,...,0.0,Son or daughter,0.0,Never married/lived together,Kalenjin,NaN,NaN,0.0,No education,0.0
370183,KELDK01,NaN,18sep2012,02oct2013,379.0,13.0,04=Diaries Interview,06apr2013,4.0,2013.0,...,0.0,Son or daughter,0.0,Never married/lived together,Kalenjin,NaN,NaN,0.0,No education,0.0


final comment on merging --> DID NOT MANAGE TO MERGE inc_ind_by_trx_yr_mo_usd_incRR and housing_conditions --> remember to include limitations of this choice in our report!

In [16]:
official.replace('.', np.NaN) # replace NAs consistently along the whole dataset
official.replace(' ', np.NaN)

,hh_ids,unique_hhs,first_trx_date_hh,last_trx_date_hh,tot_hh_daysofobs,tot_hh_monthsofobs,interview_designation,int_date,int_month,int_year,...,dem_i_age_workage,dem_i_relhead,dem_i_hhead,dem_i_marstat,dem_i_tribe,dem_i_children_tot,dem_i_children_inhh,edu_i_enrolled,edu_i_attain,edu_i_attain_yrs
0,KELDL02,NaN,03sep2012,02oct2013,394.0,13.0,04=Diaries Interview,02oct2013,10.0,2013.0,...,1.0,Household head,1.0,Married/living together,Luo,2.0,0.0,0.0,Secondary (some or complete),12.0
1,KELDL02,NaN,03sep2012,02oct2013,394.0,13.0,04=Diaries Interview,02oct2013,10.0,2013.0,...,1.0,Household head,1.0,Married/living together,Luo,2.0,0.0,0.0,Secondary (some or complete),12.0
2,KELDL02,NaN,03sep2012,02oct2013,394.0,13.0,04=Diaries Interview,02oct2013,10.0,2013.0,...,1.0,Household head,1.0,Married/living together,Luo,2.0,0.0,0.0,Secondary (some or complete),12.0
3,KELDL02,NaN,03sep2012,02oct2013,394.0,13.0,04=Diaries Interview,02oct2013,10.0,2013.0,...,1.0,Household head,1.0,Married/living together,Luo,2.0,0.0,0.0,Secondary (some or complete),12.0
4,KELDL02,NaN,03sep2012,02oct2013,394.0,13.0,04=Diaries Interview,02oct2013,10.0,2013.0,...,1.0,Household head,1.0,Married/living together,Luo,2.0,0.0,0.0,Secondary (some or complete),12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
735424,KVIHK34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,Grandchild,0.0,Never married/lived together,Luhya,NaN,NaN,0.0,No education,0.0
735425,KVIHK36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,Brother or sister in law,0.0,Never married/lived together,Luhya,NaN,NaN,1.0,Nursery/ Kindergarten,0.0
735426,KVIHK36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,Son or daughter,0.0,Never married/lived together,Luhya,NaN,NaN,NaN,NaN,NaN
735427,KVIHK36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,Son or daughter,0.0,Never married/lived together,Luhya,NaN,NaN,NaN,NaN,NaN


In [17]:
empty_cols = [col for col in official.columns if official[col].isnull().all()] # checking for empty columns 

print(empty_cols) # no empty columns

[]


In [18]:
# drop unimportant cols 

official = official.drop(columns = ['trx_yr_mo','first_trx_date_hh', 'last_trx_date_hh', 'tot_hh_daysofobs', 'tot_hh_monthsofobs', 'interview_designation', 'int_date', 'int_month', 'int_year', 'int_yr_mo', 'first_int_date', 'unique_accnts', 'unique_hm_owner', 'tot_acc_daysofobs', 'tot_acc_monthsofobs', 'trx_stdtime_days_hh', 'trx_stdtime_mnths_hh', 'trx_stdtime_days_acc', 'trx_stdtime_mnths_acc', 'trx_family_desc', 'trx_class_desc', 'trx_type_desc', 'trx_mode_code', 'trx_place_incommunity', 'trx_distance_km', 'trx_value_kes', 'trx_inkind_units', 'trx_inkind_value_kes'], axis=1)


In [19]:
# drop trx_class_code

official = official.drop(columns = ['trx_class_code'], axis = 1) # motivation: conflict with target prediction, see discussion in the report

In [20]:
# drop cols with more than 80% of NaNs

na_perc = official[official.columns[official.isnull().any()]].isnull().sum() * 100 / official.shape[0]
#print(na_perc)

nas_list = []
for i in na_perc.keys():
  if na_perc[i] > 80.0:
    nas_list.append(i)

official = official.drop(columns = nas_list) # DROP MOSTLY EMPTY COLUMNS

official.shape

(735429, 165)

In [21]:
perc = 80.0
min_count =  int(((100-perc)/100)*official.shape[1] + 1) # DROP MOSTLY EMPTY ROWS
official = official.dropna(axis=0, 
                    thresh=min_count) 

official.shape

(734904, 165)

Merged dataframe is ready to be split into train and test!

In [22]:
os.getcwd()

'/content/drive/MyDrive/GRAD-C24_Machine_Learning/MLProject_KenyaFinancial'

In [23]:
os.makedirs('clean_data', exist_ok=True)  # make dataset available as csv for machine learning preprocessing
compression_opts = dict(method='zip',
                        archive_name='official.csv')
official.to_csv('clean_data/official.zip', index=False,
          compression=compression_opts) 